In [ ]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import ast
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
input_file = "../data/raw/listings_detailed.csv"
output_file = "../data/processed/listings_fixed.csv"

rows = []

# Step 1: Read raw CSV safely
with open(input_file, "r", encoding="utf-8", errors="replace") as f:
    reader = csv.reader(f)
    for row in reader:
        rows.append(row)

# Step 2: Detect the correct number of columns (based on header)
expected_cols = len(rows[0])
print(f"Expected columns: {expected_cols}")

# Step 3: Fix rows with wrong number of columns
fixed_rows = []
for i, row in enumerate(rows):
    if len(row) != expected_cols:
        print(f"Fixing row {i+1}: had {len(row)} columns")
        if len(row) < expected_cols:
            # Pad missing columns
            row += [""] * (expected_cols - len(row))
        else:
            # Merge extra columns into the last one
            row = row[:expected_cols-1] + [",".join(row[expected_cols-1:])]
    fixed_rows.append(row)

# Step 4: Save fixed CSV
with open(output_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(fixed_rows)

# Step 5: Load into pandas
data = pd.read_csv(output_file)
print("DataFrame shape:", data.shape)
print(data.head())

In [ ]:
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)
data.isna().sum().to_csv("../data/processed/missing_summary.csv")

In [ ]:
data.price
data['price'] = data['price'].replace('[\$,]', '', regex = True).astype(float)

In [ ]:
Q1 = data["price"].quantile(0.25)   
Q3 = data["price"].quantile(0.75)   
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# outliers = data[(data["price"] < lower_bound) | (data["price"] > upper_bound)]
data = data[(data['price'] >= lower_bound) & (data['price'] <= upper_bound)]

In [ ]:
# data["price_log"] = np.log1p(data["price"])

In [ ]:
# plt.hist(data["price_log"], bins=50)
# plt.title("Distribution of Log-Transformed Prices")
# plt.show()

In [ ]:
# plt.boxplot(data["price_log"], vert=False)
# plt.title("Boxplot of Log-Transformed Prices")
# plt.show()

In [ ]:
    # stats.probplot(data["price_log"], dist="norm", plot=plt)
    # plt.show()

In [ ]:
# Percentile-based winsorization
lower_cap = data["price"].quantile(0.01)
upper_cap = data["price"].quantile(0.99)

data["price_capped"] = data["price"].clip(lower=lower_cap, upper=upper_cap)
data["price_log"] = np.log1p(data["price_capped"])

In [ ]:
plt.hist(data["price_log"], bins=50)
plt.title("Distribution of Log-Transformed Prices")
plt.show()

In [ ]:
stats.probplot(data["price_log"], dist="norm", plot=plt)
plt.show()

In [ ]:
sns.histplot(data["price_log"], kde=True)

In [ ]:
data['host_since'] = pd.to_datetime(data['host_since'], errors='coerce')
data['host_days'] = (pd.to_datetime("today") - data['host_since']).dt.days.fillna(0)

data['last_review'] = pd.to_datetime(data['last_review'], errors='coerce')
data['days_since_last_review'] = (pd.to_datetime("today") - data['last_review']).dt.days.fillna(9999)